# LAB 2b:  Prepare babyweight dataset.

**Learning Objectives**

1. Setup up the environment
1. Preprocess natality dataset
1. Augment natality dataset
1. Create the train and eval tables in BigQuery
1. Export data from BigQuery to GCS in CSV format


## Introduction 
In this notebook, we will prepare the babyweight dataset for model development and training to predict the weight of a baby before it is born.  We will use BigQuery to perform data augmentation and preprocessing which will be used for AutoML Tables, BigQuery ML, and Keras models trained on Cloud AI Platform.

In this lab, we will set up the environment, create the project dataset, preprocess and augment natality dataset, create the train and eval tables in BigQuery, and export data from BigQuery to GCS in CSV format.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/1b_prepare_data_babyweight.ipynb).

## Set up environment variables and load necessary libraries

Import necessary libraries.

In [1]:
import os

from google.cloud import bigquery

Set environment variables so that we can use them throughout the entire lab. We will be using our project ID for our bucket.

In [2]:
PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

## The source dataset

Our dataset is hosted in [BigQuery](https://cloud.google.com/bigquery/). The CDC's Natality data has details on US births from 1969 to 2008 and is a publically available dataset, meaning anyone with a GCP account has access. Click [here](https://console.cloud.google.com/bigquery?project=bigquery-public-data&p=publicdata&d=samples&t=natality&page=table) to access the dataset.

The natality dataset is relatively large at almost 138 million rows and 31 columns, but simple to understand. `weight_pounds` is the target, the continuous value we’ll train a model to predict.

## Create a BigQuery Dataset and Google Cloud Storage Bucket 

A BigQuery dataset is a container for tables, views, and models built with BigQuery ML. Let's create one called __babyweight__ if we have not already done so in an earlier lab. We'll do the same for a GCS bucket for our project too.

In [3]:
%%bash

## Create a BigQuery dataset for babyweight if it doesn't exist
datasetexists=$(bq ls -d | grep -w ) # TODO: Add dataset name

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset titled: babyweight"
    
    bq --location=US mk --dataset \
        --description "Babyweight" \
        $PROJECT:# TODO: Add dataset name
    echo "Here are your current datasets:"
    bq ls
fi
    
## Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
    
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "Here are your current buckets:"
    gsutil ls
fi

Creating BigQuery dataset titled: babyweight
Too many positional args, still have ['Add', 'dataset', 'name']
Here are your current datasets:
   datasetId    
 -------------- 
  feat_eng      
  serverlessml  
  taxifare      
Bucket exists, let's not recreate it.


Usage: grep [OPTION]... PATTERNS [FILE]...
Try 'grep --help' for more information.
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe


## Create the training and evaluation data tables

Since there is already a publicly available dataset, we can simply create the training and evaluation data tables using this raw input data. First we are going to create a subset of the data limiting our columns to `weight_pounds`, `is_male`, `mother_age`, `plurality`, and `gestation_weeks` as well as some simple filtering and a column to hash on for repeatable splitting.

* Note:  The dataset in the create table code below is the one created previously, e.g. "babyweight".

### Lab Task #1: Preprocess and filter dataset

We have some preprocessing and filtering we would like to do to get our data in the right format for training.

Preprocessing:
* Cast `is_male` from `BOOL` to `STRING`
* Cast `plurality` from `INTEGER` to `STRING` where `[1, 2, 3, 4, 5]` becomes `["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]`
* Add `hashcolumn` hashing on `year` and `month`

Filtering:
* Only want data for years later than `2000`
* Only want baby weights greater than `0`
* Only want mothers whose age is greater than `0`
* Only want plurality to be greater than `0`
* Only want the number of weeks of gestation to be greater than `0`

In [5]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data AS
SELECT
    # TODO: Add selected raw features and preprocessed features
    weight_pounds,
    CAST(is_male AS STRING) as is_male,
    mother_age,
    CASE
        WHEN plurality = 1 THEN "Single(1)"
        WHEN plurality = 2 THEN "Twins(2)"
        WHEN plurality = 3 THEN "Triplets(3)"
        WHEN plurality = 4 THEN "Quadruplets(4)"
        WHEN plurality = 5 THEN "Quintuplets(5)"
    END AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING)
        )
    ) AS hashmonth
FROM
    publicdata.samples.natality
WHERE
    # TODO: Add filters
    year > 2000 AND
    weight_pounds > 0 AND
    mother_age > 0 AND
    plurality > 0 AND
    gestation_weeks > 0

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1319.38query/s]                        


""


### Lab Task #2: Augment dataset to simulate missing data

Now we want to augment our dataset with our simulated babyweight data by setting all gender information to `Unknown` and setting plurality of all non-single births to `Multiple(2+)`.

In [9]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_augmented_data AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data
UNION ALL
SELECT
    # TODO: Replace is_male and plurality as indicated above
    weight_pounds,
    "Unknown" AS is_male,
    mother_age,
    CASE
        WHEN plurality = "Single(1)" THEN plurality
        ELSE "Multiple(2+)"
    END AS plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1252.78query/s]                        


""


### Lab Task #3: Split augmented dataset into train and eval sets

Using `hashmonth`, apply a modulo to get approximately a 75/25 train/eval split.

### Split augmented dataset into train dataset

#### **Exercise**:   **RUN** the query to create the training data table.

In [10]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_train AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    # TODO: Modulo hashmonth to be approximately 75% of the data
    ABS(MOD(hashmonth, 4)) < 3

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1115.90query/s]                        


""


### Split augmented dataset into eval dataset

#### **Exercise**:   **RUN** the query to create the evaluation data table.

In [16]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_eval AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    # TODO: Modulo hashmonth to be approximately 25% of the data
    ABS(MOD(hashmonth,4)) = 3

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1431.18query/s]                        


""


## Verify table creation

Verify that you created the dataset and training data table.


In [13]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 354.55query/s]                          
Downloading: 0rows [00:00, ?rows/s]


,weight_pounds,is_male,mother_age,plurality,gestation_weeks


In [17]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 405.72query/s]                          
Downloading: 0rows [00:00, ?rows/s]


,weight_pounds,is_male,mother_age,plurality,gestation_weeks


## Lab Task #4: Export from BigQuery to CSVs in GCS

Use BigQuery Python API to export our train and eval tables to Google Cloud Storage in the CSV format to be used later for TensorFlow/Keras training. We'll want to use the dataset we've been using above as well as repeat the process for both training and evaluation data.

In [18]:
# Construct a BigQuery client object.
client = bigquery.Client()

dataset_name = "babyweight"  # TODO: Add dataset name

# Create dataset reference object
dataset_ref = client.dataset(dataset_id=dataset_name, project=client.project)

# Export both train and eval tables
for step in [
    # TODO: Loop over train and eval
    "train",
    "eval",
]:
    destination_uri = os.path.join(
        "gs://", BUCKET, dataset_name, "data", f"{step}*.csv"
    )
    table_name = f"babyweight_data_{step}"
    table_ref = dataset_ref.table(table_name)
    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        # Location must match that of the source table.
        location="US",
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print(
        "Exported {}:{}.{} to {}".format(
            client.project, dataset_name, table_name, destination_uri
        )
    )

Exported qwiklabs-asl-03-5d5c0ba6f6d6:babyweight.babyweight_data_train to gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train*.csv
Exported qwiklabs-asl-03-5d5c0ba6f6d6:babyweight.babyweight_data_eval to gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/eval*.csv


## Verify CSV creation

Verify that we correctly created the CSV files in our bucket.

In [19]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/data/*.csv

gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/eval000000000000.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/eval000000000001.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/eval000000000002.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000000.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000001.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000002.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000003.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000004.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000005.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000006.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000007.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000008.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000009.csv
gs://qwiklabs-asl-03-5d5c0ba6f6d6/babyweight/data/train000000000010

In [20]:
%%bash
gsutil cat gs://${BUCKET}/babyweight/data/train000000000000.csv | head -5

weight_pounds,is_male,mother_age,plurality,gestation_weeks
1.4991433816,true,16,Single(1),24
1.8739292269999999,true,17,Single(1),25
3.3267755335799998,true,19,Single(1),30
4.3541296745,true,19,Single(1),30


In [21]:
%%bash
gsutil cat gs://${BUCKET}/babyweight/data/eval000000000000.csv | head -5

weight_pounds,is_male,mother_age,plurality,gestation_weeks
7.1870697412,true,15,Single(1),35
6.0009827716399995,true,15,Single(1),35
6.60504936952,true,44,Single(1),36
6.37576861704,true,44,Single(1),37


## Lab Summary: 
In this lab, we setup our environment, created a BigQuery dataset, preprocessed and augmented the natality dataset, created train and eval tables in BigQuery, and exported data from BigQuery to GCS in CSV format.

Copyright 2020 Google LLC
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.